# Funciones de agregación

En esta práctica desarrollaremos los usos más comunes de las funciones 

`groupby, agg ` y `transform` y una forma posible de agregación con datos tipo datetime



In [1]:
# Librerías
import pandas as pd
import numpy as np


# 1. Análisis inicial de datos

In [2]:
# Leemos el dataset desde el link
df = pd.read_excel("https://github.com/chris1610/pbpython/blob/master/data/sample-salesv3.xlsx?raw=True")
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [3]:
# Renombramos las columnas
df.rename(columns={'account number': 'numero_cuenta', 'name':'nombre', 'sku':'producto', 'quantity': 'cantidad',
                   'unit price':'precio_unitario', 'ext price':'total_ventas', 'date':'fecha'}, inplace=True)

Vamos a utilizar los métodos `pandas.info()` y `pandas.describe()` para conocer el dataset 

In [4]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   numero_cuenta    1500 non-null   int64  
 1   nombre           1500 non-null   object 
 2   producto         1500 non-null   object 
 3   cantidad         1500 non-null   int64  
 4   precio_unitario  1500 non-null   float64
 5   total_ventas     1500 non-null   float64
 6   fecha            1500 non-null   object 
dtypes: float64(2), int64(2), object(3)
memory usage: 82.2+ KB


1. **¿Cuáles son las observaciones?**

  Son las lineas de ventas de una companía 

2. **¿Cuáles son las variables?**

  El numero de cuenta, el nombre, el codigo de producto, la cantidad vendida, el precio unitario, el total de ventas y la fecha.

Observamos que no hay valores faltantes

In [5]:
# Describe
df.describe(include='all')

,numero_cuenta,nombre,producto,cantidad,precio_unitario,total_ventas,fecha
count,1500.000000,1500,1500,1500.000000,1500.000000,1500.000000,1500
unique,NaN,20,30,NaN,NaN,NaN,1500
top,NaN,Kulas Inc,S2-77896,NaN,NaN,NaN,2014-01-01 07:21:51
freq,NaN,94,73,NaN,NaN,NaN,1
mean,485957.841333,NaN,NaN,24.308667,55.007527,1345.856213,NaN
std,223974.044572,NaN,NaN,14.439265,25.903267,1084.914881,NaN
min,141962.000000,NaN,NaN,-1.000000,10.030000,-97.160000,NaN
25%,257198.000000,NaN,NaN,12.000000,32.500000,472.177500,NaN
50%,527099.000000,NaN,NaN,25.000000,55.465000,1050.390000,NaN
75%,714466.000000,NaN,NaN,37.000000,77.075000,2068.330000,NaN


Las personas dueñas de la compañía nos piden realizar algunos análisis de los clientes:

* ¿Qué cantidad de productos compró cada cliente?
* ¿Cuál fue el monto de venta total de cada cliente? ¿Y la venta promedio?
* ¿Hubo ventas que representaron más del 5% del monto total de ventas de un cliente?
* ¿Cómo se distribuyeron el monto de ventas durante los trimestres del año? ¿Todos los clientes tienen perfiles similares?

Para ello vamos a necesitar utilizar funciones de agregación

# 2. Función groupby




In [6]:
# Observemos la documentación de la función
? pd.DataFrame.groupby()

En nuestro caso se nos pidió realizar ciertos análisis por cliente, por lo tanto debemos indicar que queremos agrupar por esa variable en el argumento `by`

In [7]:
# Agrupamos por cliente
grupo_clientes = df.groupby(by='nombre')

Obtenemos un dataset agrupado del cual podemos observar algunos atributos relevantes como:
* **groups**: diccionario de los grupos con las categorías como llaves y los indices de las filas 
como valores
* **ngroups**: cantidad de grupos

In [8]:
# Dataset agrupado
grupo_clientes

In [9]:
# Grupos
grupo_clientes.groups

{'Barton LLC': [0, 85, 91, 96, 99, 105, 117, 125, 158, 182, 183, 195, 202, 226, 237, 268, 306, 310, 397, 420, 423, 427, 430, 456, 459, 482, 505, 516, 524, 530, 540, 605, 622, 635, 636, 646, 650, 652, 666, 667, 691, 718, 733, 740, 830, 853, 854, 895, 906, 907, 915, 920, 930, 963, 985, 989, 1008, 1015, 1017, 1019, 1031, 1071, 1077, 1078, 1081, 1094, 1108, 1132, 1147, 1149, 1158, 1175, 1208, 1218, 1257, 1321, 1330, 1346, 1350, 1414, 1422, 1473], 'Cronin, Oberbrunner and Spencer': [51, 110, 148, 168, 179, 181, 191, 197, 216, 222, 224, 252, 253, 270, 280, 292, 314, 358, 363, 374, 395, 454, 491, 495, 511, 527, 545, 556, 579, 630, 673, 680, 710, 788, 817, 848, 942, 948, 956, 973, 994, 1026, 1059, 1061, 1073, 1100, 1115, 1116, 1148, 1157, 1174, 1177, 1195, 1198, 1211, 1222, 1245, 1305, 1316, 1351, 1364, 1389, 1390, 1464, 1469, 1487, 1491], 'Frami, Hills and Schmidt': [12, 22, 24, 81, 82, 100, 217, 223, 278, 301, 319, 333, 337, 354, 359, 392, 408, 432, 485, 490, 494, 538, 564, 603, 609, 640, 65

In [10]:
# Cantidad de grupos
grupo_clientes.ngroups

20

Si queremos agrupar por más de una variable pasamos una lista con los nombres de las variables en el argumento `by`.

Aunque no es necesario para contestar las preguntas, agrupemos según cliente y producto

In [11]:
# Agrupamos por cliente y producto
grupos_clientes_producto = df.groupby(by=['nombre', 'producto'])

In [12]:
# Grupos
grupos_clientes_producto.groups

{('Barton LLC', 'B1-04202'): [1077], ('Barton LLC', 'B1-05914'): [646], ('Barton LLC', 'B1-20000'): [0, 195, 306, 1008], ('Barton LLC', 'B1-33087'): [226, 733], ('Barton LLC', 'B1-33364'): [125, 963, 1218], ('Barton LLC', 'B1-38851'): [183, 202, 427, 456, 636, 930, 989, 1017], ('Barton LLC', 'B1-50809'): [85, 423, 505, 540], ('Barton LLC', 'B1-53102'): [91, 1350, 1473], ('Barton LLC', 'B1-53636'): [516, 1108], ('Barton LLC', 'B1-65551'): [430, 524, 1071, 1149], ('Barton LLC', 'B1-69924'): [459, 605], ('Barton LLC', 'B1-86481'): [99, 1132], ('Barton LLC', 'S1-06532'): [105, 397, 622, 666, 853, 854], ('Barton LLC', 'S1-27722'): [650, 906, 1019], ('Barton LLC', 'S1-30248'): [237, 915], ('Barton LLC', 'S1-47412'): [117, 895, 1147, 1158], ('Barton LLC', 'S1-50961'): [1015, 1208], ('Barton LLC', 'S1-65481'): [1081, 1175], ('Barton LLC', 'S1-82801'): [268, 830, 920, 1321], ('Barton LLC', 'S1-93683'): [158, 1330], ('Barton LLC', 'S2-10342'): [182, 635, 691, 1078, 1414], ('Barton LLC', 'S2-1148

Observamos que al agrupar por dos columnas ahora la clave del diccionario es una tupla con dos valores: cliente y producto

In [13]:
# Cantidad de grupos
grupos_clientes_producto.ngroups

544

¿Es posible agrupar primero por una variable y luego por otra?

In [14]:
grupo_clientes.groupby(by='producto')

AttributeError: ignored

# 3. Agregación

Los procesos agregación consisten en generar información de resumen de los grupos.

**Manera directa**: consiste en la aplicación de la función de resumen de manera directa

**Método aggregate**: usar la función agg. Esta forma nos va a permitir varias cosas:
  * Aplicar distintas funciones a distintas variables
  * Aplicar funciones propias
  * Nombrar las columnas agregadas (útil por prolijidad)

## 3.1 Manera directa

Podemos utilizar distintas funciones para obtener estadísticas de resumen.

Un listado de las funciones más comunes se encuentra [aquí](https://pandas.pydata.org/docs/user_guide/groupby.html#aggregation)

Buscamos responder dos de las preguntas de interés:
* ¿Qué cantidad de productos compró cada cliente?
* ¿Cuál fue el monto de venta total de cada cliente?

In [15]:
# Aplicamos la función sum al dataframe agrupado
grupo_clientes.sum()

,numero_cuenta,cantidad,precio_unitario,total_ventas
nombre,,,,
Barton LLC,60692300,2041,4409.06,109438.50
"Cronin, Oberbrunner and Spencer",17232266,1673,3336.99,89734.55
"Frami, Hills and Schmidt",56661696,1903,3942.49,103569.59
"Fritsch, Russel and Anderson",59741550,2112,4350.41,112214.71
"Halvorson, Crona and Champlin",35046790,1284,3244.92,70004.36
Herman LLC,8801644,1538,3259.15,82865.00
Jerde-Hilpert,36693810,1999,4635.54,112591.43
"Kassulke, Ondricka and Metz",19686336,1647,3266.76,86451.07
Keeling LLC,50984594,1806,4223.63,100934.30


Esta forma nos permite contestar las preguntas. Sin embargo, es un poco desprolija ya que realiza la suma de todas las variables que son **numéricas**.

Podemos mejorar el resultado seleccionando sólo las columnas de interés.

In [16]:
# Aplicamos la función sum al dataframe agrupado seleccionando las columnas de interés
grupo_suma_clientes = grupo_clientes[['cantidad', 'total_ventas']].sum()
grupo_suma_clientes

,cantidad,total_ventas
nombre,,
Barton LLC,2041,109438.50
"Cronin, Oberbrunner and Spencer",1673,89734.55
"Frami, Hills and Schmidt",1903,103569.59
"Fritsch, Russel and Anderson",2112,112214.71
"Halvorson, Crona and Champlin",1284,70004.36
Herman LLC,1538,82865.00
Jerde-Hilpert,1999,112591.43
"Kassulke, Ondricka and Metz",1647,86451.07
Keeling LLC,1806,100934.30


Esta forma de agrupar tiene como particularidad que el o las variables de agrupación se encuentran como índice del dataset de resultado y no como columnas

In [17]:
# Indice del dataframe resultado
grupo_suma_clientes.index

Index(['Barton LLC', 'Cronin, Oberbrunner and Spencer',
       'Frami, Hills and Schmidt', 'Fritsch, Russel and Anderson',
       'Halvorson, Crona and Champlin', 'Herman LLC', 'Jerde-Hilpert',
       'Kassulke, Ondricka and Metz', 'Keeling LLC', 'Kiehn-Spinka',
       'Koepp Ltd', 'Kuhn-Gusikowski', 'Kulas Inc', 'Pollich LLC',
       'Purdy-Kunde', 'Sanford and Sons', 'Stokes LLC', 'Trantow-Barrows',
       'White-Trantow', 'Will LLC'],
      dtype='object', name='nombre')

In [18]:
# Columnas del dataframe resultado
grupo_suma_clientes.columns

Index(['cantidad', 'total_ventas'], dtype='object')

### Grupo como columnas

Si deseamos conservar el o los grupos como columnas y no como índice del dataframe que resulta de una agregación podemos utilizar dos formas:

* El método `reset_index()` luego de la agregación
* Especificando el argumento **as_index=False** en la función `groupby()` 

In [19]:
# Utilizando el método reset_index
grupo_clientes[['cantidad', 'total_ventas']].sum().reset_index()

,nombre,cantidad,total_ventas
0,Barton LLC,2041,109438.50
1,"Cronin, Oberbrunner and Spencer",1673,89734.55
2,"Frami, Hills and Schmidt",1903,103569.59
3,"Fritsch, Russel and Anderson",2112,112214.71
4,"Halvorson, Crona and Champlin",1284,70004.36
5,Herman LLC,1538,82865.00
6,Jerde-Hilpert,1999,112591.43
7,"Kassulke, Ondricka and Metz",1647,86451.07
8,Keeling LLC,1806,100934.30
9,Kiehn-Spinka,1756,99608.77


In [20]:
# Utilizando el argumento as_index
df.groupby(by='nombre', as_index=False)[['cantidad', 'total_ventas']].sum()

,nombre,cantidad,total_ventas
0,Barton LLC,2041,109438.50
1,"Cronin, Oberbrunner and Spencer",1673,89734.55
2,"Frami, Hills and Schmidt",1903,103569.59
3,"Fritsch, Russel and Anderson",2112,112214.71
4,"Halvorson, Crona and Champlin",1284,70004.36
5,Herman LLC,1538,82865.00
6,Jerde-Hilpert,1999,112591.43
7,"Kassulke, Ondricka and Metz",1647,86451.07
8,Keeling LLC,1806,100934.30
9,Kiehn-Spinka,1756,99608.77


## 3.2 Método agg(regate)

Como se mencionó el método agg es mucho más flexible y nos permite construir agregaciones más complejas y prolijas.

Existen múltiples maneras de trabajar:

1.   Funciones
2.   Diccionario {columna:funcion}
3.   Agregaciones con nombre



### 3.2.1 Aggregate con funciones

En el argumento `func` especificamos el nombre de la función de agregación como función o string.
También podemos pasar una lista de funciones.



In [21]:
# Especificando una única función de agregación
grupo_clientes.agg(func=sum)

,numero_cuenta,cantidad,precio_unitario,total_ventas
nombre,,,,
Barton LLC,60692300,2041,4409.06,109438.50
"Cronin, Oberbrunner and Spencer",17232266,1673,3336.99,89734.55
"Frami, Hills and Schmidt",56661696,1903,3942.49,103569.59
"Fritsch, Russel and Anderson",59741550,2112,4350.41,112214.71
"Halvorson, Crona and Champlin",35046790,1284,3244.92,70004.36
Herman LLC,8801644,1538,3259.15,82865.00
Jerde-Hilpert,36693810,1999,4635.54,112591.43
"Kassulke, Ondricka and Metz",19686336,1647,3266.76,86451.07
Keeling LLC,50984594,1806,4223.63,100934.30


In [22]:
# Especificando la función como string
grupo_clientes.agg(func='sum')

,numero_cuenta,cantidad,precio_unitario,total_ventas
nombre,,,,
Barton LLC,60692300,2041,4409.06,109438.50
"Cronin, Oberbrunner and Spencer",17232266,1673,3336.99,89734.55
"Frami, Hills and Schmidt",56661696,1903,3942.49,103569.59
"Fritsch, Russel and Anderson",59741550,2112,4350.41,112214.71
"Halvorson, Crona and Champlin",35046790,1284,3244.92,70004.36
Herman LLC,8801644,1538,3259.15,82865.00
Jerde-Hilpert,36693810,1999,4635.54,112591.43
"Kassulke, Ondricka and Metz",19686336,1647,3266.76,86451.07
Keeling LLC,50984594,1806,4223.63,100934.30


### 3.2.2 Diccionario {columna:funcion}

Esta forma es más flexible ya que podemos indicar distintas funciones de agregación para cada variable obteniendo las estadísticas que queremos sólo para las variables que queremos.

Al igual que antes la función se puede indicar como función o string y también es posible indicar una lista de funciones de agregación.

In [23]:
# Diccionario con una única función por columna
grupo_clientes.agg(func={'cantidad':'sum', 'total_ventas':sum})

,cantidad,total_ventas
nombre,,
Barton LLC,2041,109438.50
"Cronin, Oberbrunner and Spencer",1673,89734.55
"Frami, Hills and Schmidt",1903,103569.59
"Fritsch, Russel and Anderson",2112,112214.71
"Halvorson, Crona and Champlin",1284,70004.36
Herman LLC,1538,82865.00
Jerde-Hilpert,1999,112591.43
"Kassulke, Ondricka and Metz",1647,86451.07
Keeling LLC,1806,100934.30


In [24]:
# Diccionario con lista de funciones
grupo_clientes.agg(func={'cantidad':'sum', 'total_ventas':[sum, np.mean]})

cantidad total_ventas             
                                     sum          sum         mean
nombre                                                            
Barton LLC                          2041    109438.50  1334.615854
Cronin, Oberbrunner and Spencer     1673     89734.55  1339.321642
Frami, Hills and Schmidt            1903    103569.59  1438.466528
Fritsch, Russel and Anderson        2112    112214.71  1385.366790
Halvorson, Crona and Champlin       1284     70004.36  1206.971724
Herman LLC                          1538     82865.00  1336.532258
Jerde-Hilpert                       1999    112591.43  1265.072247
Kassulke, Ondricka and Metz         1647     86451.07  1350.797969
Keeling LLC                         1806    100934.30  1363.977027
Kiehn-Spinka                        1756     99608.77  1260.870506
Koepp Ltd                           1790    103660.54  1264.152927
Kuhn-Gusikowski                     1665     91094.28  1247.866849
Kulas Inc                           2265    137351.96  1461.191064
Pollich LLC                         1707     87347.18  1196.536712
Purdy-Kunde                         1450     77898.21  1469.777547
Sanford and Sons                    1704     98822.98  1391.872958
Stokes LLC                          1766     91535.92  1271.332222
Trantow-Barrows                     2271    123381.38  1312.567872
White-Trantow                       2258    135841.99  1579.558023
Will LLC                            1828    104437.60  1411.318919

Con este resultado podemos contestar facilmente las tres primeras preguntas que nos solicitaron:

* ¿Qué cantidad de productos compró cada cliente?
* ¿Cuál fue el monto de venta total de cada cliente? ¿Y la venta promedio?

Si bien este resultado es útil puede ser necesario mejorar la prolijidad con nombres de columnas más claros

### 3.2.3 Agregaciones con nombre

Las agregaciones con nombre permiten obtener columnas con nombres designados por nosotros. Existen dos maneras de realizarlo:

1. Mediante el uso de `pandas.NamedAgg`
2. Sintaxis "implícita"

Estas dos formas tienen como desventaja que se debe indicar las funciones de agregación de a una (no se pueden utilizar listas)

**NamedAgg**

En `NamedAgg` indicamos la columna (**column**) y la función de agregación que queremos utilizar (**aggfunc**)

In [ ]:
# Uso de NamedAgg
grupo_clientes.agg(cantidad_total=pd.NamedAgg(column="cantidad", aggfunc="sum"),
                   venta_total=pd.NamedAgg(column="total_ventas", aggfunc="sum"),
                   venta_promedio=pd.NamedAgg(column="total_ventas", aggfunc="mean"))

,cantidad_total,venta_total,venta_promedio
nombre,,,
Barton LLC,2041,109438.50,1334.615854
"Cronin, Oberbrunner and Spencer",1673,89734.55,1339.321642
"Frami, Hills and Schmidt",1903,103569.59,1438.466528
"Fritsch, Russel and Anderson",2112,112214.71,1385.366790
"Halvorson, Crona and Champlin",1284,70004.36,1206.971724
Herman LLC,1538,82865.00,1336.532258
Jerde-Hilpert,1999,112591.43,1265.072247
"Kassulke, Ondricka and Metz",1647,86451.07,1350.797969
Keeling LLC,1806,100934.30,1363.977027


**Sintaxis implícita**

Se indica en una tupla (columna,función de agregación)

In [25]:
# Sin utilizar NamedAgg (sintaxis implícita)
grupo_clientes.agg(cantidad_total =('cantidad','sum'),
                   venta_total = ('total_ventas',sum),
                   venta_promedio = ('total_ventas', np.mean))

,cantidad_total,venta_total,venta_promedio
nombre,,,
Barton LLC,2041,109438.50,1334.615854
"Cronin, Oberbrunner and Spencer",1673,89734.55,1339.321642
"Frami, Hills and Schmidt",1903,103569.59,1438.466528
"Fritsch, Russel and Anderson",2112,112214.71,1385.366790
"Halvorson, Crona and Champlin",1284,70004.36,1206.971724
Herman LLC,1538,82865.00,1336.532258
Jerde-Hilpert,1999,112591.43,1265.072247
"Kassulke, Ondricka and Metz",1647,86451.07,1350.797969
Keeling LLC,1806,100934.30,1363.977027


# 4. Transform

Los procesos apply de transformación consisten en la aplicación de una función o transformación de datos a nivel grupo sin reducir la cantidad de registros del dataset.

En este caso vamos a trabajar en la creación de nuevas variables usando información de los grupos. En particular, buscaremos contestar la pregunta:

* ¿Hubo ventas que representaron más del 5% del monto total de ventas de un cliente?

Para ello es necesario:

1. Calcular el monto total de ventas por cliente
2. Expresar cada venta como un porcentaje del total
3. Filtrar los datos de manera acorde a la pregunta

La sintaxis que vamos a utilizar es:

```
df['columna'].transform(funcion)
```


In [ ]:
# Calculamos las ventas totales para cada cliente
grupo_clientes['total_ventas'].transform('sum')

0       109438.50
1       123381.38
2       137351.96
3        86451.07
4       112591.43
          ...    
1495    112214.71
1496    103569.59
1497     91535.92
1498     87347.18
1499    104437.60
Name: total_ventas, Length: 1500, dtype: float64

Observemos que nos el método `transform` nos devuelve una serie de 1500 elementos (cantidad de registros del dataset)

In [ ]:
# Creamos una columna con la venta total por cliente
df['venta_total_cliente'] = grupo_clientes['total_ventas'].transform('sum')

In [ ]:
# Observamos el resultado
df[['nombre', 'total_ventas', 'venta_total_cliente']].sample(10)

,nombre,total_ventas,venta_total_cliente
524,Barton LLC,1528.36,109438.50
437,Herman LLC,1866.48,82865.00
651,Kiehn-Spinka,3799.74,99608.77
942,"Cronin, Oberbrunner and Spencer",2303.25,89734.55
335,Sanford and Sons,977.84,98822.98
777,Keeling LLC,919.23,100934.30
503,Kulas Inc,1048.44,137351.96
884,Herman LLC,3733.92,82865.00
429,Koepp Ltd,938.20,103660.54
404,Jerde-Hilpert,535.50,112591.43


In [ ]:
# Creamos una variable que indica para cada venta de un cliente 
# qué porcentaje de su venta total representa 
df['porcentaje_ventas_cliente'] = (df.total_ventas/df.venta_total_cliente)*100

In [ ]:
# Observamos el resultado
df[['nombre', 'total_ventas', 'venta_total_cliente', 'porcentaje_ventas_cliente']].sample(10)

,nombre,total_ventas,venta_total_cliente,porcentaje_ventas_cliente
294,Kiehn-Spinka,96.07,99608.77,0.096447
1221,White-Trantow,4251.78,135841.99,3.129945
1006,Koepp Ltd,1891.12,103660.54,1.824339
1184,"Fritsch, Russel and Anderson",992.80,112214.71,0.884732
96,Barton LLC,181.82,109438.50,0.166139
211,Sanford and Sons,2205.00,98822.98,2.231262
86,Kuhn-Gusikowski,110.91,91094.28,0.121753
504,White-Trantow,2707.38,135841.99,1.993036
627,Keeling LLC,3318.90,100934.30,3.288179
690,Herman LLC,736.80,82865.00,0.889157


In [ ]:
# Conservamos las ventas que representan más de un 5% de la venta total del cliente
df.query("porcentaje_ventas_cliente>5")[['nombre', 'producto', 'total_ventas',
                                         'venta_total_cliente', 'porcentaje_ventas_cliente']]

,nombre,producto,total_ventas,venta_total_cliente,porcentaje_ventas_cliente
90,"Halvorson, Crona and Champlin",S2-11481,3578.27,70004.36,5.111496
476,"Halvorson, Crona and Champlin",S1-47412,4011.04,70004.36,5.729700
781,Kuhn-Gusikowski,B1-04202,4770.72,91094.28,5.237124
852,"Halvorson, Crona and Champlin",S1-93683,4174.72,70004.36,5.963514
900,Pollich LLC,S1-93683,4621.04,87347.18,5.290428
932,Purdy-Kunde,B1-20000,4414.62,77898.21,5.667165
1032,"Kassulke, Ondricka and Metz",S1-27722,4418.47,86451.07,5.110949
1042,Purdy-Kunde,B1-38851,4046.29,77898.21,5.194330


# 5. Trabajo con fechas

Para el trabajo con fechas vamos a utilizar el objeto `pandas.Grouper`. Recordemos que en el Grouper debemos especificar la columna (**key**) y la frecuencia de agrupación (**freq**). 

Un listado exhaustivo de las frecuencias se encuentra [aquí](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) 

Las preguntas para responder respecto al tiempo eran:

¿Cómo se distribuyeron el monto de ventas durante los trimestres del año? ¿Todos los clientes tienen perfiles similares en ventas por cuatrimestre?

In [ ]:
# Convertimos la fecha a datetime
df['fecha'] = pd.to_datetime(df.fecha)

In [ ]:
# Indicamos freq=Q para indicar agrupación cuatrimestral 
df.groupby(by=pd.Grouper(key='fecha', freq='Q')).agg(ventas_totales=('total_ventas',sum))

,ventas_totales
fecha,
2014-03-31,535494.66
2014-06-30,514081.99
2014-09-30,513883.87
2014-12-31,455323.80


In [ ]:
# Agrupamos por cliente y trimestre
df.groupby(['nombre', pd.Grouper(key='fecha', freq='Q')]).agg(ventas_totales=('total_ventas',sum))

ventas_totales
nombre                          fecha                     
Barton LLC                      2014-03-31        21909.13
                                2014-06-30        32158.10
                                2014-09-30        38345.55
                                2014-12-31        17025.72
Cronin, Oberbrunner and Spencer 2014-03-31        26809.63
...                                                    ...
White-Trantow                   2014-12-31        34248.39
Will LLC                        2014-03-31        44405.86
                                2014-06-30        20371.41
                                2014-09-30        16803.35
                                2014-12-31        22856.98

[80 rows x 1 columns]